In [1]:
%config Completer.use_jedi = False

In [2]:
import numpy as np
import os
import tensorflow as tf
import json
from tqdm import tqdm
import random
from scipy.spatial.distance import cdist
from pathlib import Path
from multiprocessing import Pool

In [3]:
def config_gpus():
    gpus = tf.config.list_physical_devices('GPU')
    if gpus:
        try:
            # Currently, memory growth needs to be the same across GPUs
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
                logical_gpus = tf.config.list_logical_devices('GPU')
                print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
        except RuntimeError as e:
            # Memory growth must be set before GPUs have been initialized
            print(e)
    return

In [4]:
def parse_fn(example_proto):
    features = {"data": tf.io.FixedLenFeature((), tf.string),
                "label": tf.io.FixedLenFeature((), tf.int64),
                }
    parsed_features = tf.io.parse_single_example(example_proto, features)
    data = tf.io.decode_raw(parsed_features["data"], tf.float32)
    data = tf.reshape(data, shape=[100, 5])
    return data, parsed_features["label"]


def load_tfrecord(filepath, batch_size=128, shuffle=True):
    AUTOTUNE = tf.data.experimental.AUTOTUNE
    dataset = tf.data.TFRecordDataset(filepath, num_parallel_reads=4)
    dataset = dataset.map(parse_fn, num_parallel_calls=4)
    dataset = dataset.batch(batch_size)
    if shuffle:
        dataset = dataset.shuffle(buffer_size=100)
    return dataset.prefetch(AUTOTUNE)

In [5]:
def evaluate(model_path, data_path, metric, threshold):
    
    model = tf.keras.models.load_model(model_path)
    
    test_dataset = load_tfrecord(filepath=list(map(str, Path(data_path).glob("**/*.tfrecord"))), batch_size=600, shuffle=False)
    
    for batch in tqdm(test_dataset):
        x_test, y_test = batch
        embeds = model(x_test, training=False).numpy()
        y_test = y_test.numpy()
        
        for y in np.unique(y_test):
                        
            positive_embeds = embeds[y_test == y]
            negative_embeds = embeds[y_test != y]
            
            positive_dist = cdist(positive_embeds, positive_embeds, metric).round(4)
            negative_dist = cdist(positive_embeds, negative_embeds, metric).round(4)

            positive_dist = np.reshape(positive_dist, (-1,))
            positive_dist = positive_dist[positive_dist >= 1e-6]
            negative_dist = np.reshape(negative_dist, (-1,))
            
            if len(positive_dist) < 2 or len(negative_dist) < 2:
                continue
            
            fn = np.sum(negative_dist < threshold) / len(negative_dist) 
            fp = np.sum(positive_dist > threshold) / len(positive_dist)

            log = {"id": int(y),
                   "fp": float(np.mean(fp) * 100),
                   "fn": float(np.mean(fn) * 100)}

            dist_log = {"id": int(y),
                        "d_positive": positive_dist.tolist(),
                        "d_negative": negative_dist.tolist()}

            with open("eval_{}_threshold_{}.json".format(metric, threshold), "a") as f:
                f.write(json.dumps(log)+"\n")

#             with open("eval_{}_distance.json".format(metric), "a") as f:
#                 f.write(json.dumps(dist_log)+"\n")
        
        print(log)
                
    return

In [6]:
evaluate(model_path="ckpt/2022-06-03-18h16m56s/serving/", data_path="data/test/", metric="cosine", threshold=0.4)

1it [00:02,  2.09s/it]

{'id': 166022, 'fp': 10.0, 'fn': 0.10084033613445378}


2it [00:02,  1.00s/it]

{'id': 166045, 'fp': 0.0, 'fn': 0.0}


3it [00:02,  1.52it/s]

...

{'id': 167954, 'fp': 47.61904761904761, 'fn': 0.0}


86it [00:40,  1.89it/s]

{'id': 167978, 'fp': 9.523809523809524, 'fn': 0.481811611659841}


87it [00:40,  2.14it/s]

{'id': 167999, 'fp': 0.0, 'fn': 0.0}


In [15]:
import pandas as pd

with open("eval_cosine_threshold_0.4.json") as f:
    df = [json.loads(line) for line in f]
    
df = pd.DataFrame(df)
len(df)
df

,id,fp,fn
0,161000,9.523810,0.120453
1,161001,0.000000,0.000000
2,161002,0.000000,0.457721
3,161003,0.000000,0.000000
4,161004,14.285714,0.096362
...,...,...,...
8153,167995,0.000000,0.196541
8154,167996,0.000000,0.029621
8155,167997,39.285714,0.444313
8156,167998,0.000000,0.000000


In [16]:
df['fp'].mean(), df['fn'].mean()

(5.772894602877442, 0.14282834537919326)